In [7]:
import numpy as np
import pandas as pd
from transformers import BartForConditionalGeneration, AutoTokenizer
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

In [2]:
import os 
os.path.exists("../code/model_finetuning/outputs")

True

In [16]:
model = BartForConditionalGeneration.from_pretrained("../code/model_finetuning/outputs")
model = model.to("cuda")

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [8]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

In [29]:
input_text = "paraphrase though it left the drinker smoking at the ears for several hours afterward."

In [30]:
tokenized_input = tokenizer(input_text, truncation=True, padding='max_length', return_tensors='pt').to("cuda")

In [33]:
outputs = model.generate(**tokenized_input, max_length=200)

In [34]:
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [35]:
print(f"{input_text}\n\n{output_text}")

paraphrase though it left the drinker smoking at the ears for several hours afterward.

Paraphrase though it was, it left the drinker smoking for several hours afterwards.


In [28]:
print(f"{input_text}\n\n{output_text}")

paraphrase October arrived, spreading a damp chill over the grounds and into the castle. Madam Pomfrey, the nurse, was kept busy by a sudden spate of colds among the staff and students. Her Pepperup potion worked instantly, though it left the drinker smoking at the ears for several hours afterward. Ginny Weasley, who had been looking pale, was bullied into taking some by Percy. The steam pouring from under her vivid hair gave the impression that her whole head was on fire. 

October arrived, spreading a damp chill over the grounds and into the castle. Madam Pomfrey, the nurse, was busy with a sudden outbreak of colds among the staff and students. Her Pepperup worked instantly, though it left the drinker smoking at the ears for several hours afterwards. Ginny Weasley, who had been looking pale, was forced to take some by Percy. The steam pouring from under her vivid hair made it appear that her whole head was on fire.


In [7]:
model_args = Seq2SeqArgs()
model_args.eval_batch_size = 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_seq_length = 128
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.train_batch_size = 8
model_args.use_multiprocessing = False

model_args.do_sample = True
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.max_length = 128
model_args.top_k = 50
model_args.top_p = 0.95

In [8]:
model = Seq2SeqModel(encoder_type="bart", 
                     encoder_decoder_name="../code/model_finetuning/outputs",
                     args=model_args
                    )


# model_reloaded = Seq2SeqModel(
#     "roberta",
#     encoder_decoder_name="outputs",
#     args=model_args,
# )

OSError: Can't load the configuration of '../code/model_finetuning/outputs\encoder'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '../code/model_finetuning/outputs\encoder' is the correct path to a directory containing a config.json file